In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import os
import statsmodels.formula.api as smf

11.1 Statsmodel 
----



In [2]:
os.chdir(os.path.join('..','data'))
df = pd.read_csv(r'2002FemPreg.csv',low_memory=False)
df = df[df['outcome']==1] # live births
df = df.dropna(subset=['agepreg','totalwgt_lb'])

In [3]:
age_preg = df['agepreg']
baby_wgt = df['totalwgt_lb']

In [4]:
formula = 'totalwgt_lb ~ agepreg'
# first the dependent variable then the independent variables in the above formula
model = smf.ols(formula,data=df) 
results = model.fit()

In [5]:
inter = results.params['Intercept']
slope = results.params['agepreg']

In [6]:
# check the significance level of the parameters
results.pvalues

Intercept    0.000000e+00
agepreg      5.722947e-11
dtype: float64

The values are quite significant, indicating there is some correlation between them

In [7]:
results.rsquared

0.004738115474710369

And results provides resid, a sequence of residuals, and fittedvalues, a
sequence of fitted values corresponding to agepreg

In [8]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Wed, 28 Nov 2018   Prob (F-statistic):           5.72e-11
Time:                        10:32:57   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Std(ys) is the standard deviation of the dependent variable, which is the
RMSE if you have to guess birth weights without the benefit of any explanatory variables. Std(res) is the standard deviation of the residuals, which
is the RMSE if your guesses are informed by the mother’s age. As we have
already seen, knowing the mother’s age provides no substantial improvement
to the predictions.

11.2 Multiple Regression
---

In chapter 4 we saw that first babies tend to be lighter than others, and
this effect is statistically significant. But it is a strange result because there
is no obvious mechanism that would cause first babies to be lighter. So we
might wonder whether this relationship is spurious.

In fact, there is a possible explanation for this effect. We have seen that
birth weight depends on mother’s age, and we might expect that mothers of
first babies are younger than others

In [9]:
first = df[df['birthord']==1]
other = df[df['birthord']!=1] 
diff_wgt = np.mean(first['totalwgt_lb']) - np.mean(other['totalwgt_lb'])
diff_age = np.mean(first['agepreg']) - np.mean(other['agepreg'])
diff_wgt,diff_age

(-0.12476118453549034, -3.558721516986065)

In [10]:
slope*diff_age

-0.062113396786983674

The predicted value of the totalwgt by the known mother's age is about 50% of the actual.
So we conclude, tentatively, that the observed difference in birth weight can be **partly
explained** by the difference in mother’s age.

In [11]:
# explore the variables
# adding a new variable which will be catergorial
df['isfirst'] = df['birthord']==1
formula2 = 'totalwgt_lb ~ isfirst'
results2 = smf.ols(formula2,data=df).fit()

In [12]:
results2.params['isfirst[T.True]']*diff_age

0.4439903118911115

The slope and the intercept are statistically significant, which means that
they were unlikely to occur by chance, but the the R2 value for this model
is small, which means that isfirst doesn’t account for a substantial part of
the variation in birth weight.

But now we can
fit a single model that includes both variables. With the formula
totalwgt_lb ~ isfirst + agepreg, we get: 

In [13]:
formula3 = 'totalwgt_lb ~ agepreg + isfirst'
results3 = smf.ols(formula3,data=df).fit()

R2 for this model is a little higher, which indicates that the two variables
together account for more variation in birth weight than either alone (but
not by much)

11.2 Non-linear Relationships
---

Remembering that the contribution of agepreg might be nonlinear, we might
consider adding a variable to capture more of this relationship. One option
is to create a column, agepreg2, that contains the squares of the ages

In [14]:
df['agepreg2'] = np.square(df['agepreg'])

In [15]:
formula4 = 'totalwgt_lb ~ agepreg + agepreg2 + isfirst'
results4 = smf.ols(formula4,data=df).fit()

In [16]:
#summary

model = ['agepreg','isfirst','isfirst+agepreg','isfirst+agepreg+agepreg2']
res = [results.rsquared,results2.rsquared,results3.rsquared,results4.rsquared]

pd.DataFrame(res,columns=['r-squared'],index=model)

,r-squared
agepreg,0.004738
isfirst,0.001960
isfirst+agepreg,0.005289
isfirst+agepreg+agepreg2,0.007462


11.4 Data Mining
---

But the R2 values of those
models is very low, which means that they have little predictive power. In
this section we’ll try to do better.

Well, the NSFG dataset includes 244 variables about
each pregnancy and another 3087 variables about each respondent. Maybe
some of those variables have predictive power. To find out which ones are
most useful, why not try them all?

totalwgt_lb = agepreg + "some_variable"
We will explore for more variables to see if the variables has a significant prediction power. 

In [17]:
resp = pd.read_csv('respondent.csv')

In [18]:
df2 = df[df['prglngth']>30]
df_joined = df2.join(resp,on='caseid',rsuffix='_r')

In [19]:
df_joined.head()

,Unnamed: 0,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,...,pubassis_i_r,basewgt_r,adj_mod_basewgt_r,finalwgt_r,secu_r,sest_r,cmintvw_r,cmlstyr,screentime,intvlngth
0,0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0.0,2335.279149,2846.79949,4744.19135,2.0,18.0,1233.0,1221.0,16:30:59,64.294000
1,1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0.0,2335.279149,2846.79949,4744.19135,2.0,18.0,1233.0,1221.0,16:30:59,64.294000
2,2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,...,0.0,2335.279149,2846.79949,4744.19135,2.0,18.0,1234.0,1222.0,18:19:09,75.149167
3,3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0.0,2335.279149,2846.79949,4744.19135,2.0,18.0,1234.0,1222.0,18:19:09,75.149167
4,4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,0.0,2335.279149,2846.79949,4744.19135,2.0,18.0,1234.0,1222.0,18:19:09,75.149167


In [20]:
r_squares = {}
for name in df_joined.columns:
    try:
        if df_joined[name].var() < 1e-7:
            continue

        formula = 'totalwgt_lb ~ agepreg +'+name
        model = smf.ols(formula,data=df_joined)
        if model.nobs < len(df_joined)/2:
            continue
            
        result = model.fit()
        
    except (ValueError, TypeError,SyntaxError):
        continue
        
    r_squares.setdefault(name,result.rsquared)
        

In [21]:
sorted(r_squares.items(),key=lambda x:x[1],reverse=True)

[('totalwgt_lb', 1.0),
 ('birthwgt_lb', 0.9498127305978009),
 ('lbw1', 0.3008240784470769),
 ('prglngth', 0.13012519488625063),
 ('wksgest', 0.12340041363361054),
 ('agecon', 0.10203149928156052),
 ('mosgest', 0.027144274639579802),
 ('babysex', 0.01855092529394209),
 ('race', 0.016199503586253106),
 ('nbrnaliv', 0.016017752709788113),
 ('birthwgt_oz', 0.013102457615706165),
 ('anynurse', 0.012529022541810764),
 ('bfeedwks', 0.01219368840449575),
 ('rmarout6', 0.011354138472805753),
 ('hisprace', 0.011238349302030826),
 ('fmarcon5', 0.0106049646842995),
 ('fmarout5', 0.01046169136737718),
 ('fmarital', 0.009944942659110834),
 ('evuseint', 0.00993306080712264),
 ('pubassis', 0.009858545642850935),
 ('pmarpreg', 0.009840804911715795),
 ('poverty', 0.009743158975296984),
 ('stopduse', 0.009315099704133023),
 ('whentell', 0.009056250355562567),
 ('rmarital', 0.008267774071422207),
 ('tellfath', 0.008089600034942523),
 ('matchfound', 0.008066317368677134),
 ('marend02_i', 0.0080412538958868

In [22]:
for i in range(10):
    if i % 2==0:
        continue
    print(str(i)+' is Odd number')

1 is Odd number
3 is Odd number
5 is Odd number
7 is Odd number
9 is Odd number


This approach, is called data mining. An advantage of
data mining is that it can discover unexpected patterns. A hazard is that
many of the patterns it discovers are either random or spurious. 

This way we can explore other variables for the model to increase the prediction power.

11.5 Prediction
----

After sorting the results on the basis of R2
we now select the variables for our model to increase the predictive power

variables - 

prglngth

The first useful predictive variable is babysex which indicates whether the
baby is male or female.

Next is race, which indicates whether the respondent is white, black, or
other

The next variable on the list is nbrnaliv, which indicates whether the pregnancy yielded multiple births.

Next on the list is paydu, which indicates whether the respondent owns
her home.


This is called data mining. An advantage of
data mining is that it can discover unexpected patterns. A hazard is that
many of the patterns it discovers are either random or spurious.

In [40]:
formula = ('totalwgt_lb ~ agepreg + C(race) + babysex==1 + paydu==1 + nbrnaliv>1 + totincr')
results_new = smf.ols(formula,data=df_joined).fit()

results_new.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.052
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     42.36
Date:                Wed, 28 Nov 2018   Prob (F-statistic):           1.49e-58
Time:                        10:47:43   Log-Likelihood:                -8915.3
No. Observations:                5415   AIC:                         1.785e+04
Df Residuals:                    5407   BIC:                         1.790e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept                6.7074      0.092     73.110      0.000       6.528       6.887
C(race)[T.2]             0.3858      0.041      9.498      0.000       0.306       0.465
C(race)[T.3]             0.2208      0.069      3.206      0.001       0.086       0.356
babysex == 1[T.True]     0.2936      0.034      8.592      0.000       0.227       0.361
paydu == 1[T.True]       0.0344      0.037      0.931      0.352      -0.038       0.107
nbrnaliv > 1[T.True]    -1.3585      0.134    -10.155      0.000      -1.621      -1.096
agepreg                  0.0116      0.003      3.720      0.000       0.006       0.018
totincr                 -0.0059      0.005     -1.256      0.209      -0.015       0.003
==============================================================================
Omnibus:                      267.459   Durbin-Watson:                   1.568
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              989.467
Skew:                          -0.027   Prob(JB):                    1.38e-215
Kurtosis:                       5.093   Cond. No.                         212.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

totincr is encoded numerically from 1-14, with each increment representing
about $5000 in annual income. 

So we can treat these values as numerical,
expressed in units of $5000

observations - 
1. People who own a house have their babies 0.34 lbs heavier
2. Male babies tend to be 0.29 lbs heavier
3. encoding 1 for black, 2 for white and 3 for others - 
black babies are lighter than others by 0.22 - 0.38 lbs
4. Twins are lighter than 1.4 lbs

Above all the r2 is quite small. RMSE of the model and the RMSE of actual data is almost same. So knowing all the above parameters wont help much in predicting the totalwgt of the baby.

In [52]:
print('Model std(residual) / RMSE: ',np.std(results_new.resid), ' | Actual: ',np.std(df_joined['totalwgt_lb']))

Model std(residual) / RMSE:  1.2554098374447706  | Actual:  1.2711444893867523


11.6 Logistic Regression
-------

log(o) = β0 + (β1) x1 + (β2) x2 + Error

Where o is the odds in favor of a particular outcome; in the example, o would
be the odds of having a boy.

odds = p/(1-p)

11.7 Estimating the parameters
-----

Unlike linear regression, logistic regression does not have a closed form solution, so it is solved by guessing an initial solution and improving it iteratively.

In [23]:
y = np.array([0,1,0,1])
x1 = np.array([0,0,0,1])
x2 = np.array([0,1,1,1])
# initial guess
beta = [-1.5, 2.8,1.1]

In [24]:
log_o = beta[0] + beta[1]*x1 + beta[2]*x2
log_o

array([-1.5, -0.4, -0.4,  2.4])

In [25]:
o = np.exp(log_o)  # odds
p = o/(o+1)  # probability
o,p

(array([ 0.22313016,  0.67032005,  0.67032005, 11.02317638]),
 array([0.18242552, 0.40131234, 0.40131234, 0.9168273 ]))

The likelihood of an outcome is p when y==1 and 1-p when y==0.

In [26]:
likes = y*p+(1-y)*(1-p)
np.prod(likes)

0.1800933529673034

This is the joint probability for the model. 
For these values of beta, the likelihood of the data is 0.18. The
goal of logistic regression is to find parameters that maximize this likelihood. To do that, most statistics packages use an iterative solver
like Newton’s method 
(see https://en.wikipedia.org/wiki/Logistic_regression#Model_fitting)

11.8 Implementation of logistic regression
--------------------
logistic regression using statsmodel library


In [27]:
df2['boy']  = (df2['babysex']==1).astype(int)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [28]:
df2.head()

,Unnamed: 0,caseid,pregordr,howpreg_n,howpreg_p,moscurrp,nowprgdk,pregend1,pregend2,nbrnaliv,...,basewgt,adj_mod_basewgt,finalwgt,secu_p,sest,cmintvw,totalwgt_lb,isfirst,agepreg2,boy
0,0,1,1,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,3410.389399,3869.349602,6448.271112,2,9,NaN,8.8125,True,1099.5856,1
1,1,1,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,3410.389399,3869.349602,6448.271112,2,9,NaN,7.8750,False,1540.5625,0
2,2,2,1,NaN,NaN,NaN,NaN,5.0,NaN,3.0,...,7226.301740,8567.549110,12999.542260,2,12,NaN,9.1250,True,205.3489,1
3,3,2,2,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,7226.301740,8567.549110,12999.542260,2,12,NaN,7.0000,False,317.9089,0
4,4,2,3,NaN,NaN,NaN,NaN,6.0,NaN,1.0,...,7226.301740,8567.549110,12999.542260,2,12,NaN,6.1875,False,335.9889,0


In [29]:
model5 = smf.logit('boy ~ agepreg',data=df2)
results = model5.fit()
results.summary()

Optimization terminated successfully.
         Current function value: 0.693022
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8781
Model:                          Logit   Df Residuals:                     8779
Method:                           MLE   Df Model:                            1
Date:                Wed, 28 Nov 2018   Pseudo R-squ.:               5.361e-06
Time:                        10:33:39   Log-Likelihood:                -6085.4
converged:                       True   LL-Null:                       -6085.5
                                        LLR p-value:                    0.7984
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0067      0.098      0.068      0.946      -0.186       0.199
agepreg        0.0010      0.004      0.255      0.798      -0.007       0.009
==============================================================================
"""

The parameter of agepreg is positive, which suggests that older mothers are
more likely to have boys, but the p-value is 0.783, which means that the
apparent effect could easily be due to chance

In [30]:
formula6 = 'boy ~ agepreg + hpagelb + birthord + C(race)'
results6 = smf.logit(formula6,data=df2).fit() 
results6.summary()

Optimization terminated successfully.
         Current function value: 0.692916
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 8686
Model:                          Logit   Df Residuals:                     8680
Method:                           MLE   Df Model:                            5
Date:                Wed, 28 Nov 2018   Pseudo R-squ.:               0.0001953
Time:                        10:33:39   Log-Likelihood:                -6018.7
converged:                       True   LL-Null:                       -6019.8
                                        LLR p-value:                    0.7988
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.0333      0.105     -0.318      0.750      -0.238       0.172
C(race)[T.2]    -0.0257      0.051     -0.501      0.616      -0.126       0.075
C(race)[T.3]    -0.0069      0.084     -0.082      0.935      -0.172       0.158
agepreg         -0.0035      0.006     -0.630      0.529      -0.014       0.007
hpagelb          0.0052      0.004      1.225      0.221      -0.003       0.014
birthord         0.0113      0.022      0.510      0.610      -0.032       0.055
================================================================================
"""

None of the estimated parameters are statistically significant. The pseudo-R2
value is a little higher, but that could be due to chance

In [33]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])

In [35]:
actual  = df2['boy']
predicted = results.predict()>=0.5
true_positive = predicted*actual
true_negative = (1-predicted)*(1-actual)
acc = sum(true_positive + true_negative)/len(actual)
acc

0.5078009338344153

In [37]:
actual.mean()   #baseline

0.5078009338344153

The result is same as the baseline, 0.507. But, we should
not take this result too seriously. We used the same data to build and test
the model, so the model may not have predictive power on new data. 

Instead we should test the data with test and cross-validation set.

In [38]:
# prediction using the current model 
columns = ['agepreg','hpagelb','birthord','race']
input_value = pd.DataFrame([[35,39,3,2]],columns=columns)
y = results.predict(input_value)
y

0    0.510263
dtype: float64